In [1]:
# 협업 필터링에는 2가지가 있음

# 1. 고객별 개개인 추천시스템 (사용자 추천 서비스)
# 2. 일반적으로 관련된 상품 추천 (아이템 추천 서비스)

In [2]:
# 고객을 전부 넣어야 하는 이유

# 고객이 상품을 하나만 구매할 때 해당 상품이 더 많이 등장한다는 사실로부터 우리는 다음과 같은 정보를 얻을 수 있습니다.

# 상품의 인기도: 해당 상품이 다른 상품들보다 인기가 높다면, 고객이 단 하나의 상품을 구매할 때에도 해당 상품이 더 많이 등장할 가능성이 높아집니다. 
# 이는 해당 상품이 고객들에게 매력적인 제품이라는 증거가 됩니다.

# 고객의 취향: 고객이 단 하나의 상품을 선택할 때 해당 상품이 다른 상품들보다 더 많이 등장한다면,
# 이는 해당 상품이 고객의 취향에 맞는 제품이라는 증거가 됩니다. 이를 통해 상품을 더욱 정확하게 타겟팅하여 마케팅 전략을 수립할 수 있습니다.

########
# 크로스셀링 기회: 단 하나의 상품만을 구매하는 고객에게 해당 상품과 관련된 다른 상품들을 제안함으로써 추가적인 구매 기회를 창출할 수 있습니다.  
# 예를 들어, 만약 해당 상품이 어떤 다른 제품과 함께 사용하는 것이 일반적이라면, 해당 제품과 함께 구매할 만한 크로스셀링 기회가 있을 수 있습니다.

In [3]:
# 한 개만 구매하는 고객을 빼는 경우

# 한 개만 구매하는 고객을 빼고 연관분석을 수행하는 경우는 "단일 구매자 분석"이라고도 불리며, 이 경우 다음과 같은 정보를 얻을 수 있습니다.

# 상품 번들링 기회: 연관분석을 통해 다른 고객들이 함께 구매하는 상품들을 파악함으로써, 해당 상품과 함께 구매되는 다른 상품들을 제안할 수 있습니다. 
# 이를 통해 상품 번들링 기회를 찾아내어 매출을 높일 수 있습니다.

# 상품 포지셔닝: 연관분석 결과를 통해 어떤 상품들이 함께 구매되는지 파악함으로써, 해당 상품의 위치와 경쟁 업체와의 차별화 전략을 수립할 수 있습니다.

# 상품 품질 개선: 연관분석을 통해 함께 구매되는 상품들 중에 불만족스러운 상품이 있다면, 해당 상품의 품질을 개선하여 고객 만족도를 높일 수 있습니다.

# 고객 세분화: 단일 구매자 분석을 통해 한 개만 구매하는 고객들의 구매 패턴을 파악할 수 있습니다. 
# 이를 기반으로 고객을 세분화하여 타겟팅을 더욱 정확하게 할 수 있습니다.

In [4]:
# 1개짜리도 포함시킨 이유

# 만약, 여러 번 구매하지만 상품 하나만 구매하는 경우가 다른 고객들과 함께 구매하는 상품들의 교집합이 크다면,
# 들을 제외하면 추천시스템이 불완전해질 수 있습니다. 따라서, 이러한 경우에는 여러 번 구매하지만 상품 하나만 구매하는 경우도 모델에 포함시켜야 합니다.

# 다른 고객들과 함께 구매하는 상품들의 교집합이 크다: 여러 번 구매하지만 상품 하나만 구매하는 경우가 다른 고객들과 함께 구매하는 상품들이 많이 겹친다


# 그러나, 여러 번 구매하지만 상품 하나만 구매하는 경우가 다른 고객들과 함께 구매하는 상품들의 교집합이 작거나 없는 경우에는 
# 이들을 모델에서 제외시켜도 괜찮습니다. 이러한 경우에는 다른 고객들과 구매 패턴이 다르기 때문에,
# 이들의 데이터를 모델에 포함시키면 노이즈로 작용하여 추천 정확도를 떨어뜨릴 수 있습니다.

In [5]:
import pandas as pd
import numpy as np 
import matplotlib
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')
matplotlib.rc("font",family = "NaNumGothic")
matplotlib.rc("axes",unicode_minus = False) # 음수표시 

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


# 연령별

In [6]:
df= pd.read_csv("../data/구매상품데이터프레임 20대.csv",index_col=0)

records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) \
                    for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])


te = TransactionEncoder()
te_ary = te.fit(records).transform(records, sparse=True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)


association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005,
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    if violation == 0 or ex_violation == 0:
        collective_strength = 0
    else:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation+1)+1
    collective_strengths.append(collective_strength)
    
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities

df_association = association_rules_df.sort_values(by='lift',ascending=False)
df_association

df_association.to_csv('../data/연관규칙 20대.csv', encoding="utf-8-sig")

Processing 267 combinations | Sampling itemset size 32


In [7]:
df= pd.read_csv("../data/구매상품데이터프레임 30대.csv",index_col=0)

records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) \
                    for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])


te = TransactionEncoder()
te_ary = te.fit(records).transform(records, sparse=True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)


association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005,
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    if violation == 0 or ex_violation == 0:
        collective_strength = 0
    else:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation+1)+1
    collective_strengths.append(collective_strength)
    
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities

df_association = association_rules_df.sort_values(by='lift',ascending=False)


df_association

df_association.to_csv('../data/연관규칙 30대.csv', encoding="utf-8-sig")

Processing 273 combinations | Sampling itemset size 32


In [8]:
df= pd.read_csv("../data/구매상품데이터프레임 40대.csv",index_col=0)

records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) \
                    for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])


te = TransactionEncoder()
te_ary = te.fit(records).transform(records, sparse=True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)


association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005,
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    if violation == 0 or ex_violation == 0:
        collective_strength = 0
    else:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation+1)+1
    collective_strengths.append(collective_strength)
    
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities

df_association = association_rules_df.sort_values(by='lift',ascending=False)
df_association

df_association.to_csv('../data/연관규칙 40대.csv', encoding="utf-8-sig")

Processing 363 combinations | Sampling itemset size 32


In [9]:
df= pd.read_csv("../data/구매상품데이터프레임 50대.csv",index_col=0)

records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) \
                    for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])


te = TransactionEncoder()
te_ary = te.fit(records).transform(records, sparse=True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)


association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005,
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    if violation == 0 or ex_violation == 0:
        collective_strength = 0
    else:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation+1)+1
    collective_strengths.append(collective_strength)
    
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities

df_association = association_rules_df.sort_values(by='lift',ascending=False)
df_association

df_association.to_csv('../data/연관규칙 50대.csv', encoding="utf-8-sig")

Processing 597 combinations | Sampling itemset size 32


In [10]:
df= pd.read_csv("../data/구매상품데이터프레임 60대 이상.csv",index_col=0)

records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) \
                    for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])


te = TransactionEncoder()
te_ary = te.fit(records).transform(records, sparse=True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)


association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005,
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    if violation == 0 or ex_violation == 0:
        collective_strength = 0
    else:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation+1)+1
    collective_strengths.append(collective_strength)
    
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities

df_association = association_rules_df.sort_values(by='lift',ascending=False)
df_association

df_association.to_csv('../data/연관규칙 60대 이상.csv', encoding="utf-8-sig")

Processing 459 combinations | Sampling itemset size 32


# 연령별 비교

In [14]:
df_20= pd.read_csv("../data/연관규칙 20대.csv",index_col=0)
df_30= pd.read_csv("../data/연관규칙 30대.csv",index_col=0)
df_40= pd.read_csv("../data/연관규칙 40대.csv",index_col=0)
df_50= pd.read_csv("../data/연관규칙 50대.csv",index_col=0)
df_60= pd.read_csv("../data/연관규칙 60대 이상.csv",index_col=0)

In [17]:
df_20['antecedents'] = df_20['antecedents'].str.replace('frozenset','')
df_20['consequents'] = df_20['consequents'].str.replace('frozenset','')
df_20 = df_20[['antecedents','consequents','lift']]
df_30['antecedents'] = df_30['antecedents'].str.replace('frozenset','')
df_30['consequents'] = df_30['consequents'].str.replace('frozenset','')
df_30 = df_30[['antecedents','consequents','lift']]
df_40['antecedents'] = df_40['antecedents'].str.replace('frozenset','')
df_40['consequents'] = df_40['consequents'].str.replace('frozenset','')
df_40 = df_40[['antecedents','consequents','lift']]
df_50['antecedents'] = df_50['antecedents'].str.replace('frozenset','')
df_50['consequents'] = df_50['consequents'].str.replace('frozenset','')
df_50 = df_50[['antecedents','consequents','lift']]
df_60['antecedents'] = df_60['antecedents'].str.replace('frozenset','')
df_60['consequents'] = df_60['consequents'].str.replace('frozenset','')
df_60 = df_60[['antecedents','consequents','lift']]


In [19]:
result = pd.concat([df_20, df_30, df_40, df_50, df_60], axis=1)
result

,antecedents,consequents,lift,antecedents,consequents,lift,antecedents,consequents,lift,antecedents,consequents,lift,antecedents,consequents,lift
19,({'양파'}),({'당근'}),1.508483,({'오이'}),({'유정란/친환경'}),0.748312,({'유정란/친환경'}),({'당근'}),0.864176,({'유정란/친환경'}),({'당근'}),0.824755,({'중파'}),({'유정란/친환경'}),0.923496
18,({'당근'}),({'양파'}),1.508483,({'유정란/친환경'}),({'오이'}),0.748312,({'당근'}),({'유정란/친환경'}),0.864176,({'당근'}),({'유정란/친환경'}),0.824755,({'유정란/친환경'}),({'중파'}),0.923496
39,({'유정란/친환경'}),({'구운도시락김'}),1.167927,({'백미/유'}),({'유정란/친환경'}),0.750800,({'시금치'}),({'두부'}),0.920806,({'두부'}),({'대파'}),0.815532,({'두부'}),({'양배추'}),0.698674
38,({'구운도시락김'}),({'유정란/친환경'}),1.167927,({'유정란/친환경'}),({'백미/유'}),0.750800,({'두부'}),({'시금치'}),0.920806,({'대파'}),({'두부'}),0.815532,({'양배추'}),({'두부'}),0.698674
27,({'당근'}),({'콩나물'}),1.138070,({'유정란/친환경'}),({'유기농우유'}),0.823561,({'두부'}),({'찌개용두부'}),0.508570,({'두부'}),({'찌개용두부'}),0.532862,({'유정란/친환경'}),({'양배추'}),0.847283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,({'백미/유'}),({'유정란/친환경'}),0.732548,({'양파'}),({'오이'}),1.058318
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,({'두부'}),({'애호박'}),0.725454,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,({'애호박'}),({'두부'}),0.725454,NaN,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,({'백미/유'}),({'두부'}),0.673721,NaN,NaN,NaN


In [ ]:
result.to_csv('../data/연관규칙 연령별 비교.csv', encoding="utf-8-sig")